# Creating a new planner in nuPlan <a name="introduction"></a>


## Setup

In [1]:
# Useful imports
import os
import hydra
import nest_asyncio
from IPython.core.display import display, HTML
from bokeh.io import output_notebook


/tmp/ipykernel_10391/4095267831.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
nest_asyncio.apply()
output_notebook()
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

# Simulating the planner <a name="simulation"></a>

In [3]:
%pwd

'/host_home/Repos/nuplan-devkit/experiments'

In [4]:
%env NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
%env NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps

env: NUPLAN_DATA_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset
env: NUPLAN_MAPS_ROOT=/home/ehdykhne/Repos/Datasets/nuplan/dataset/maps


## Prepare the simulation config

In [5]:
from tutorials.utils.tutorial_utils import construct_simulation_hydra_paths

# Location of paths with all simulation configs
BASE_CONFIG_PATH = os.path.join(os.getenv('NUPLAN_TUTORIAL_PATH', ''), '../nuplan/planning/script')
simulation_hydra_paths = construct_simulation_hydra_paths(BASE_CONFIG_PATH)

# Create a temporary directory to store the simulation artifacts

DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database (2.5h of 8 autolabeled logs in Las Vegas)
    'scenario_filter=one_continuous_log',  # simulate only one log
    "scenario_filter.log_names=['2021.06.14.16.48.02_veh-12_04057_04438']",
    'scenario_filter.limit_total_scenarios=1',  # use 1 total scenarios
]
ckpt_dir = '/home/ehdykhne/Repos/nuplan-devkit/experiments/pretrained_checkpoints/pdm_offset_checkpoint.ckpt'
#'/home/sacardoz/checkpoints/urbandriver_checkpoint.ckpt'
#"/home/sacardoz/tutorial_vector_framework/training_simple_vector_experiment/train_default_simple_vector/2023.11.23.09.55.21/best_model/epoch.ckpt"
#"/home/sacardoz/training_raster_experiment/train_default_raster/2023.11.23.07.36.36/best_model/epoch.ckpt"
# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=simulation_hydra_paths.config_path)

# Compose the configuration
cfg = hydra.compose(config_name=simulation_hydra_paths.config_name, overrides=[
    '+simulation=closed_loop_reactive_agents',
    #'model=pgm_hybrid_model',
    'planner=pdm_hybrid_planner',
    f"planner.pdm_hybrid_planner.checkpoint_path={ckpt_dir}" ,
    #'planner.ml_planner.model_config=${model}',
    #f'planner.ml_planner.checkpoint_path={ckpt_dir}',
    #f'observation=idm_agents_observation',
    #'observation.model_config=${model}',
    #f'observation.checkpoint_path={ckpt_dir}',
    'worker=sequential',
    '+occlusion=true',
    '+occlusion.manager_type=wedge', #options: [range, shadow, wedge]
    "hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]",
    *DATASET_PARAMS,
])

## Launch simulation (within the notebook)

In [6]:
from nuplan.planning.script.run_simulation import build_simulation_runners
from nuplan.common.actor_state.tracked_objects_types import AGENT_TYPES, STATIC_OBJECT_TYPES, TrackedObjectType

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
runners, common_builder, cfg = build_simulation_runners(cfg)

Global seed set to 0
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback...
INFO:nuplan.planning.script.builders.main_callback_builder:Building MultiMainCallback: 4...DONE!


2023-11-30 22:11:11,533 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-30 22:11:11,534 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: Sequential
2023-11-30 22:11:11,535 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 1
Number of GPUs per node: 0
Number of threads across all nodes: 1
2023-11-30 22:11:11,535 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-11-30 22:11:11,535 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:32}  Building experiment folders...
2023-11-30 22:11:11,535 INFO {/home/ehdykhne/Repos/nuplan-devkit/nuplan/planning/script/builders/folder_builder.py:35}  

	Folder where all results are stored: /home/ehdykh

In [7]:
runner = runners[0]

In [8]:
runner.simulation.callback.on_simulation_start(runner.simulation.setup)

# Initialize all simulations
runner._initialize()

while runner.simulation.is_simulation_running():
    # Execute specific callback
    runner.simulation.callback.on_step_start(runner.simulation.setup, runner.planner)

    # Perform step
    planner_input = runner._simulation.get_planner_input()

    # Execute specific callback
    runner._simulation.callback.on_planner_start(runner.simulation.setup, runner.planner)

    # Plan path based on all planner's inputs
    trajectory = runner.planner.compute_trajectory(planner_input)

    # Propagate simulation based on planner trajectory
    runner._simulation.callback.on_planner_end(runner.simulation.setup, runner.planner, trajectory)

    iteration = runner.simulation._time_controller.get_iteration()

    runner.simulation.propagate(trajectory)

    # Execute specific callback
    runner.simulation.callback.on_step_end(runner.simulation.setup, runner.planner, runner.simulation.history.last())

runner.simulation.callback.on_simulation_end(runner.simulation.setup, runner.planner, runner.simulation.history)

elapsed time: 0.21787452697753906
elapsed time: 0.21610069274902344
elapsed time: 0.21921157836914062
elapsed time: 0.21982741355895996
elapsed time: 0.22512292861938477
elapsed time: 0.22797179222106934
elapsed time: 0.2278118133544922
elapsed time: 0.23047089576721191
elapsed time: 0.22745537757873535
elapsed time: 0.2280113697052002
elapsed time: 0.2236926555633545
elapsed time: 0.30179500579833984
elapsed time: 0.23219728469848633
elapsed time: 0.26755309104919434
elapsed time: 0.2407701015472412
elapsed time: 0.23062610626220703
elapsed time: 0.2515895366668701
elapsed time: 0.2322394847869873
elapsed time: 0.25140976905822754
elapsed time: 0.23848199844360352
elapsed time: 0.247786283493042
elapsed time: 0.23015666007995605
elapsed time: 0.2531614303588867
elapsed time: 0.2475576400756836
elapsed time: 0.25527215003967285
elapsed time: 0.22973108291625977
elapsed time: 0.2283937931060791
elapsed time: 0.23528170585632324
elapsed time: 0.2268679141998291
elapsed time: 0.2329523563

In [9]:
from tutorials.utils.tutorial_utils import visualize_history
visualize_history(runner.simulation._history, runner.scenario, bokeh_port=5006)

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.2)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)
